# Projeto

**Objetivo**: Desenvolver um CRUD utilizando um arquivo JSON para "persistir" os dados e aplicar algumas outras funcionalidades que seriam interessantes no sistema.

**Requisitos**: O projeto deve conter as seguintes funcionalidades/ características:

- Utilizar JSON para persistir os dados
- Função de criar o arquivo JSON (amanda) OK
- Função para adicionar registros (através do *input*, ou de um conjunto de dados históricos, por exemplo) (ricardo)
    - Não deixar para o usuário decidir o id, apenas coloque o proximo valor disponível
- Função para realizar o *udpate* (atualização) das informações (leticia)
    - Selecionar pelo ID
- Função para excluir registros (adriely)
    - Selecionar pelo ID
- Função para realizar a leitura de registros (printar os dados na tela do consumidor x, lista) (leticia)
- Função para filtrar informações dos dados (Amanda) +/- OK
- Função para aplicar alguma regra de negócio (varia de acordo com o sistema, exemplo, grupo criou um sistema de consulta de colaboradores, logo deseja-se ter uma função que calcula o faturamento de todos os funcionários que são vendedores) (lorrany)
    - Utilizar as funções de filtro e de estatística
- Função que retorne alguma estatística dos dados (lorrany)
- menu (daniel)
- Exemplos de funcionamento de cada uma das funções acima

**Pontos importantes**:

- Organização de código no ``jupyter notebook`` (não precisa de ppt, no dia apenas o notebook será necessário)
- Definição sobre o que será o sistema (livre e definido pelo grupo)
- Apresentação do sistema e funcionalidades (assim como os exemplos). Não precisa ser o grupo todo apresentando
- Gerenciamento de erros, como no caso de num input o usuário colocar 'abc' num campo de valor inteiro
- A nota do grupo será a nota de cada integrante
- Tempo de apresentação: de 15 a 20 minutos
- A princípio todos integrantes devem entregar o mesmo notebook no sistema do LMS, para fins de retorno de nota
- Não precisa se ater somente aos 10 requisitos acima, pode ir além :)

banco de dados: 
https://www.kaggle.com/datasets/iamsouravbanerjee/customer-shopping-trends-dataset

Pacotes necessários:
(**favor instalar!**)
- ``os``
- ``translate``
- ``json``
- ``csv``
- ``langdetect``

## Função para validação de respostas do usuário

In [19]:
def validacao_respostas(pergunta, respostas_validas):
  """
  Validação de respostas. A função valida respostas de um caracter
  É passado como parâmetro:
    - A pergunta que se quer fazer (str)
    - Uma lista com as respostas válidas

  Caso o usuário digite algo diferente das respostas válidas,
  a função repetirá a pergunta até que o usuário responda um valor válido.
  E só retornará opções válidas de valores
  """
  print(pergunta)
  resp = input().upper()[0]# apenas 1ªletra
  pergunta = 'Resposta inválida! ' + pergunta
  while not resp in respostas_validas:
    print(pergunta)
    resp = input().upper()[0]# apenas 1ªletra
    #resp = input(pergunta).upper()[0]# apenas 1ªletra
  return resp

## Função que lista os arquivos disponíveis

In [20]:
import os
def arquivos_disponiveis(extensao: str):
    """Função recebe uma extensão de arquivo e lista todos os arquivos com a extensão que estão no diretório

    Args:
        extensao (str): extensão do arquivo

    Returns:
        opcoes (dict): as opções de arquivos disponíveis
    """
    pergunta = 'Informe o numero do arquivo que deseja importar'
    opcoes = dict()
    arquivos = os.listdir() # lista todos os arquivos presentes na pasta
    arquivos_extensao = [arquivo for arquivo in arquivos if arquivo.endswith(extensao)] # filtra pela extensao
    print(f'Lista de arquivos {extensao.upper()} disponíveis')
    for i in range(len(arquivos_extensao)):
        print(f'\t{i+1}\t{arquivos_extensao[i]}')
        opcoes.update({str(i+1):arquivos_extensao[i]})
    return opcoes
        

## Função para a leitura de dados de um csv ou json e transformação em uma lista de dicionários

In [21]:
import json
import csv
def importar_dados(extensao:str):
    """ realiza a importação dos dados de arquivos .json ou .csv

    Args:
        extensao (str): extensão do arquivo

    Returns:
        dados(list): lista de dicionários com as variáveis selecionadas para o estudo de caso
    """
    dados = list()
    
    opcoes = arquivos_disponiveis(extensao)
    respostas_validas = list(opcoes.keys())
    pergunta = 'Informe o numero do arquivo que deseja importar'
    str_arquivo = opcoes[validacao_respostas(pergunta, respostas_validas)]
    
    if extensao == '.json':
        with open(str_arquivo, "r", encoding = "utf-8") as f:
            dados = json.load(f) # os arquivos importados via .json já estarão filtrados pelas categorias filtradas pela importação via .csv

    else:
        with open(str_arquivo, "r") as arquivo:
            arquivo_csv = csv.reader(arquivo, delimiter=",")
            next(arquivo_csv)  # pula a primeira linha (cabeçalho)
            for linha in enumerate(arquivo_csv):
                Customer_ID, Age, Gender, Item_Purchased,Category, Purchase_Amount, Location, Size, Color, Season, Review_Rating, Subscription_Status, Payment_Method, Shipping_Type, Discount_Applied, Promo_Code_Used, Previous_Purchases, Preferred_Payment_Method, Frequency_of_Purchases = linha[1]
                #print('ID', Customer_ID,'Idade', Age,'Genero', Gender,'Item',Item_Purchased,
                #                          'Categoria', Category,'Metodo_Pagamento', Payment_Method)
                # 
                dados.append({      'ID': int(Customer_ID), 
                                    'Idade': int(Age), #numérica
                                    'Genero': Gender, #categórica
                                    'Item':Item_Purchased, #categórica
                                    'Categoria': Category, #categórica
                                    'Preco': float(Purchase_Amount), #numérica
                                    'Metodo_Pagamento': Payment_Method, #categórica
                                    'Qtd_Compras': int(Previous_Purchases)#, #numérica
                                    #'Frequencia_Compra': Frequency_of_Purchases #categórica
                                    })
    return dados


## Função para traduzir palavras
É necessário instalar o pacote ``translate``

In [22]:
# (terminal) pip install translate
from translate import Translator
def traduz_palavras(lista_de_palavras):
  """
  Recebe uma lista de palavras em inglês e retorna um dicionário onde 
  a chave é a palavra da lista e o valor é a tradução em português.

  Args:
    lista_de_palavras: Uma lista de palavras em inglês.

  Returns:
    Um dicionário.
  """
  dicionario_de_traducao = {}
  traducao = Translator(to_lang='pt-br', from_lang='english')
  for palavra in lista_de_palavras:
    dicionario_de_traducao[palavra] = traducao.translate(palavra)
  return dicionario_de_traducao


## Função que retorna uma lista de valores sem repetição

In [23]:
def lista_de_valores_sem_repeticoes(lista_de_dicionarios, chave):
  """
  Recebe uma lista de dicionários.
  A função retorna lista dos valores encontrados para a chave, sem repetições de valores.

  Args:
    lista_de_dicionarios: Uma lista de dicionários.
    chave: A chave a ser pesquisada.

  Returns:
    Uma lista.
  """
  lista_de_valores = set()
  for dicionario in lista_de_dicionarios:
    if chave in dicionario:
      lista_de_valores.add(dicionario[chave])
  return list(lista_de_valores)

## Traduzindo os valores retirados do CSV

In [24]:
def traduzindo_lista_dict(lista: list):
    """Traduz os valores de uma lista de dicionários

    Args:
        lista (list): lista de dicionários

    Returns:
        lista (list): lista de dicionários traduzida
    """
    for chave in list(lista[0].keys()):
        if type(lista[0][chave]) == str:
            categorias = lista_de_valores_sem_repeticoes(lista, chave)
            dict_traducao = traduz_palavras(categorias)
            for i in range(len(lista)):

                lista[i][chave] = dict_traducao[lista[i][chave]]
    return lista

## Detectar idioma

In [25]:
from langdetect import detect 

def detectar_idioma(dados:list):
    """ A função detecta o idioma dos dados importados e
    questiona ao usuário que ser traduzí-los.
    Se a resposta for positiva, a função traduzindo_lista_dict() será chamada.

    Args:
        dados (list): dados dispostos em listas de dicionários

    Returns:
        dados (list): dados dispostos em listas de dicionários traduzidos para português
    """
    if detect(dados[0]['Metodo_Pagamento']) == 'en': # um exemplo de dados categóricos para verificar o idioma
        pergunta = 'Os dados estão em inglês, deseja traduzir?[S] Sim  [N] Não\t'
        resp = validacao_respostas(pergunta, ['S','N'])
        if resp == 'S':
            print("Essa operação pode demorar, aguarde.")
            return traduzindo_lista_dict(dados)



# Listagem de dados

In [26]:
def listar_dados(dados: (list)):
    print("\t".join(list(dados[0].keys())))
    for j in range(len(dados)):
        lista_linha = [str(list(dados[0].values())[i]) for i in range(len(list(dados[j].values())))]
        print("\t".join(lista_linha))


## Função para exportar uma lista de dicionários para um arquivo .json

In [27]:
import json
import os.path
def exportar_json(lista_vendas: list):
    """
    A função exporta a lista_vendas para um arquivo.
    O arquivo será nomeado pelo usuário
        Caso o arquivo já exista na pasta, o usuário será questionado se quer sobrescrever.
    O arquivo será no formato .json
    Será importado na mesma pasta que se encontra o script
    Args:
        lista_vendas (list): lista de dicionários com as informações de venda
    """
    nome_arquivo = input("Informe o nome do arquivo a ser criado:\t").lower()
    

    # retirando os possíveis espaços do nome
    qtd_espaco = nome_arquivo.count(' ')
    if qtd_espaco > 0:
        nome_arquivo =  nome_arquivo.replace(" ", "_", qtd_espaco)
    nome_arquivo += '.json'
    
    if os.path.exists(nome_arquivo) == True:
        pergunta = 'Já existe um arquivo com esse nome.\nDeseja sobrescrever?[S]  Sim\t [N] Não\t'
        resp_sobrescrever = validacao_respostas(pergunta, ['S','N'])
        if resp_sobrescrever == 'N':
            print('O arquivo não foi modificado.')
            return
    try:
        json_vendas = json.dumps(lista_vendas, indent=2,ensure_ascii=False)
        arquivo = open(nome_arquivo, 'w', encoding = "utf-8") 
        arquivo.write(json_vendas) 
        arquivo.close()
        print("Arquivo exportado com sucesso!")
    except:
        print("Erro ao exportar arquivo")
    return

# Verifica se a variável é um número

In [28]:
def validacao_numero(pergunta:str, var_tipo: type):
    """Valida e converte variáveis numéricas

    Args:
        pergunta (str): Pergunta realizada pelo usuário para coletar o valor
        var_tipo (type): tipo de variável numérica do valor que será informado pelo usuário

    Returns:
        valor validado e convertido de acordo com a variável var_tipo
    """
    valido = False
    print(pergunta)
    while valido != True:
        numero = input(pergunta)
        try:
            float(numero)
            valido = True
        except ValueError:
            print("Informe um valor válido!")
            numero = input(pergunta)
            valido = False
        else:
            if var_tipo == int:
                return int(numero)
            else:
                return float(numero)

## Função de filtro

In [29]:
def filtro(lista: list, categoria:str, valor_categoria, tipo:str):
    """Função que realiza os filtros de acordo com os parâmetros de entrada passados

    Args:
        lista (list): dados dispostos em listas de dicionários
        categoria (str): chave da categoria que se deseja filtrar 
        valor_categoria: valor utilizado com o operador comparativo
        tipo (str): nome do operador comparativo

    Returns:
        filtro (list): dados filtrados dispostos em listas de dicionários
    """

    if   tipo == 'Maior que':
        filtro = list(filter(lambda x: x[categoria] >  valor_categoria, lista))
    elif tipo == 'Maior ou igual':
        filtro = list(filter(lambda x: x[categoria] >= valor_categoria, lista))
    elif tipo == 'Menor que':
        filtro = list(filter(lambda x: x[categoria] <  valor_categoria, lista))
    elif tipo == 'Menor ou igual':
        filtro = list(filter(lambda x: x[categoria] <= valor_categoria, lista))
    elif tipo == 'Igual a':
        filtro = list(filter(lambda x: x[categoria] == valor_categoria, lista))
    else:       # Diferente de
        filtro = list(filter(lambda x: x[categoria] != valor_categoria, lista))
    return filtro


### Filtro Categórico

In [30]:
def filtro_categorico(dados:list, chave:str):
    """A função coleta do usuário qual operação se deseja realizar no filtro de variáveis categóricas

    Args:
        dados (list): dados dispostos em listas de dicionários
        chave (str): chave da categoria que se deseja filtrar
        As opções de filtros são
                - Diferente de
                - Igual a
    Returns:
        dados   (list): dados dispostos em listas de dicionários
        chave    (str): chave da categoria que se deseja filtrar  
        valor    (str): valor para realizar a operação de comparação 
        operacao (str): Nome da operação
    """

    pergunta = f'Informe o número da opção você quer filtrar dentro de {chave}: '
    opcoes = lista_de_valores_sem_repeticoes(dados, chave)

    respostas_validas = list(range(1,len(opcoes)+1))
    respostas_validas = list(map(str, respostas_validas))
    print(f'\nLista de opções dentro da categoria {chave}')
    for i in range(len(opcoes)):
        print(1+i,'\t',opcoes[i])

    resp = validacao_respostas(pergunta, respostas_validas)
    valor = opcoes[int(resp)-1]
    pergunta = f'''Como deseja filtrar os dados dentro de {chave}
        [1] Para valores diferentes de {valor}
        [2] Para valores iguais a de {valor}.'''
    resp = validacao_respostas(pergunta, ['1','2'])

    if resp == '1':
        operacao = 'Diferente de'
    else:
        operacao = 'Igual a'
    return dados, chave, valor, operacao

### Filtro Numérico

In [31]:
def filtro_numerico(dados:list, chave:str, var_tipo: type):
    """A função coleta do usuário qual operação se deseja realizar no filtro de variáveis numéricas

    Args:
        dados (list): dados dispostos em listas de dicionários
        chave (str): chave da categoria que se deseja filtrar
        var_tipo(type): tipo da variável (int ou float) que será utilizada para a comparação
        As opções de filtros são
                - Diferente de
                - Igual a
                - Maior que
                - Maior ou Igual
                - Menor que
                - Menor ou Igual 
    Returns:
        dados   (list): dados dispostos em listas de dicionários
        chave    (str): chave da categoria que se deseja filtrar  
        valor  (float): valor para realizar a operação de comparação 
        operacao (str): Nome da operação
    """
    operadores_comparacao = ['Maior que','Maior ou igual',
                            'Menor que','Menor ou igual',
                            'Igual a','Diferente de']
    respostas_validas = list(range(1,len(operadores_comparacao)+1))
    respostas_validas = list(map(str, respostas_validas))

    for i in range(len(respostas_validas)):
        print(1+i,'\t',operadores_comparacao[i])
    pergunta = f'Informe o número da opção do operador para usar na categoria {chave}: '
    resp = validacao_respostas(pergunta, respostas_validas)
    operacao = operadores_comparacao[int(resp)-1]
        
    pergunta = 'Informe um valor para realizar a comparação: '
    valor = validacao_numero(pergunta, var_tipo)
    print(chave, valor, operacao)
    return dados, chave, valor, operacao

# Menu filtros

In [32]:
def opcoes_filtros(dados:list):
    """A função coleta do usuário as informações necessárias para realizar a filtragem de dados.
        Informações coletadas:
        A categoria (chave) que se quer usar para realizar o filtro.
        A operação que se deseja realizar será coletada a partir de duas funções
        - filtro_categorico para variáveis categóricas
        - filtro_numerico para variáveis numéricas (float e int)
        E o valor necessário para tal.
    Args:
        dados (list): dados dispostos em listas de dicionários
    """
    tamanho = len(list(dados[0].keys())[1:])
    lista_categorias = list(dados[0].keys())[1:]
    respostas_validas = list(range(1,tamanho+1))
    respostas_validas = list(map(str, respostas_validas))

    print('\nCategorias Disponíveis para consulta:')
    for i in range(tamanho):
        print(1+i,'\t',lista_categorias[i])
    pergunta = 'Informe o número da categoria que deseja filtrar:'
    resp = validacao_respostas(pergunta, respostas_validas)
    chave = str(lista_categorias[int(resp)-1])

    if type(dados[0][chave]) == str:
        dados, chave, valor, operacao = filtro_categorico(dados, chave)
          
    elif type(dados[0][chave]) == int: 
        dados, chave, valor, operacao = filtro_numerico(dados, chave, int)
    else: # float
        dados, chave, valor, operacao = filtro_numerico(dados, chave, int)
        
    print(f'''Detalhes do filtro:
              Categoria: {chave}
              Valor:     {valor}
              Operação:  {operacao}
              ''')
    dados_filtrados = filtro(dados, chave, valor, operacao)
    
    pergunta = '''O que deseja fazer com os dados filtros:
    [1] Exibir na tela
    [2] Exportar
    '''
    resp = validacao_respostas(pergunta, ['1','2'])
    if resp == '1':
        listar_dados(dados_filtrados)
    else:
        exportar_json(dados_filtrados)

# Estatísticas

## Função para calcular ticket médio

In [33]:
def calcular_ticket_medio(dados):
    """
    O ticket médio é um indicador usado para avaliar o gasto médio dos clientes.
    Para calculá-lo, deve-se dividir o faturamento total das vendas pelo número de vendas do intervalo de tempo em questão.
    """
    faturamento = 0
    for item in dados:
        faturamento = sum([faturamento, item["Preco"]])
    
    transacoes = len(dados)

    if transacoes == 0:
        # Para evitar divisões por zero
        return 0
    else:
        ticket_medio = faturamento / transacoes
        return ticket_medio

## Regra de Negócio

In [34]:
def grupo_clientes(dados):
    """
    A clusterização de clientes permite agrupá-los por alguma semelhança.
    É um meio para entender o perfil de consumo dos clientes para que, de forma estratégica,
      sejam criados ofertas personalizadas.
    Nessa função os clientes serão separados em dois grupos:
    - A: perfil de consumo a partir do ticket médio
    - B: consome abaixo do ticket médio
    """
    for itens in dados:
        if Preco >= calcular_ticket_medio(dados):
            adicionar_registro("Grupo" : "A")
        else:
            adicionar_registro("Grupo" : "B")

SyntaxError: invalid syntax (2204531749.py, line 12)

# Função Principal

In [35]:
def main():
    
    pergunta = """Que tipo de arquivo deseja importar:
      1 \t CSV
      2 \t JSON
      3 \t Não importar nada   """
    resp =  validacao_respostas(pergunta, ['1','2','3'])
    if   resp == '1':
        dados = importar_dados('.csv')
        detectar_idioma(dados)
    elif resp == '2':
        dados = importar_dados('.json')
        detectar_idioma(dados)
    else:
        dados = list()
        
    continua = 'S'
    while continua =='S':
        pergunta = """Qual ação deseja realizar?
Para adicionar um registro             --- A
Para remover um registro               --- R
Para editar informações                --- E
Para leitura de registro               --- L
Para mostrar a regra de negócio        --- N
Para realizar estatística              --- S
Para realizar filtro                   --- F
Para exportar dados para JSON          --- J
Para sair do menu                      --- Q
        """
        opcao = validacao_respostas(pergunta, ['A','R','E','L','N','S','F','J','Q'])
        if   opcao == 'A': #ricardo
            pass
        elif opcao == 'R': #leticia
            pass
        elif opcao == 'E': #adriely
            pass
        elif opcao == 'L': #leticia
            pass
        elif opcao == 'N': #lorrany
            pass
        elif opcao == 'S': #lorrany
            pass
        elif opcao == 'F':
            opcoes_filtros(dados)
        elif opcao == 'J':
            exportar_json(dados)
        else:
            continua = 'N'
    print('Sistema Encerrado!')   
main()
    

Que tipo de arquivo deseja importar:
      1 	 CSV
      2 	 JSON
      3 	 Não importar nada   
Lista de arquivos .CSV disponíveis
	1	shopping_trends.csv
Informe o numero do arquivo que deseja importar
Os dados estão em inglês, deseja traduzir?[S] Sim  [N] Não	
Essa operação pode demorar, aguarde.


RuntimeError: generator raised StopIteration